# Experimentation
1. ## Data Ingestion

In [61]:
import os
import pandas as pd
import streamlit as st
import numpy as np

In [12]:
pwd

'C:\\Users\\Omistaja\\OneDrive\\Documents\\data_RedZone\\workspace_data\\project_insurance_charge'

In [13]:
FILE_NAME = "insurance.csv"
DATA_FOLDER = "data"

In [14]:
main_path = os.getcwd()

In [19]:
main_path

'C:\\Users\\Omistaja\\OneDrive\\Documents\\data_RedZone\\workspace_data\\project_insurance_charge'

In [20]:
file_path =os.path.join(os.path.join(main_path, DATA_FOLDER),FILE_NAME)

In [21]:
insurance_data =pd.read_csv(file_path)

In [22]:
insurance_data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# Data Inspection

In [23]:
insurance_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


# Exploration Data Analysis
1. Statistical analysis

## Model Building

In [39]:
### Train Test Split

### Usually 0.3, 0.7

In [40]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

In [41]:
X = insurance_data[[ "age", "sex", "bmi", "children", "smoker", "region"]]

In [42]:
y = insurance_data["charges"]

In [43]:
X_train, X_test, y_train, y_train = train_test_split( X, y, test_size=0.33, random_state=42)

In [44]:
X_train.shape, X_test.shape

((896, 6), (442, 6))

## Train Model

In [45]:
train_data, test_data = train_test_split( insurance_data, test_size=0.33, random_state=42)

In [46]:
train_data.shape, test_data.shape

((896, 7), (442, 7))

In [47]:
label = 'charges'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count      896.000000
mean     13379.688250
std      12110.713713
min       1121.873900
25%       4819.842913
50%       9556.297050
75%      17092.916520
max      62592.873090
Name: charges, dtype: float64


In [48]:
save_path = 'model'
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "model\"
AutoGluon Version:  0.6.2
Python Version:     3.9.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
Train Data Rows:    896
Train Data Columns: 6
Label Column: charges
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (62592.87309, 1121.8739, 13379.68825, 12110.71371)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    867.71 MB
	Train Data (Original)  Memory Usage: 0.19 MB (0.0% of available memory)
	Inferring data type of each feature based on column 

In [49]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-4965.510788,0.100936,26.278721,0.000999,0.509684,2,True,12
1,LightGBM,-5040.531574,0.007997,0.448720,0.007997,0.448720,1,True,4
2,CatBoost,-5048.357888,0.003996,17.870932,0.003996,17.870932,1,True,6
3,RandomForestMSE,-5103.436805,0.074953,0.821491,0.074953,0.821491,1,True,5
4,XGBoost,-5164.956518,0.013992,0.669585,0.013992,0.669585,1,True,9
5,LightGBMXT,-5173.916806,0.006997,0.359774,0.006997,0.359774,1,True,3
6,ExtraTreesMSE,-5194.262922,0.071955,0.804503,0.071955,0.804503,1,True,7
7,LightGBMLarge,-5337.109204,0.006997,0.953408,0.006997,0.953408,1,True,11
8,NeuralNetFastAI,-5444.887902,0.017989,2.377528,0.017989,2.377528,1,True,8
9,NeuralNetTorch,-5596.321460,0.012991,6.627894,0.012991,6.627894,1,True,10


## Model Evaluation

In [50]:
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,age,sex,bmi,children,smoker,region
764,45,female,25.175,2,no,northeast
887,36,female,30.020,0,no,northwest
890,64,female,26.885,0,yes,northwest
1293,46,male,25.745,3,no,northwest
259,19,male,31.920,0,yes,northwest


In [51]:
save_model_predictor = TabularPredictor.load(save_path)

In [52]:
y_pred = save_model_predictor.predict(test_data_nolab)

In [53]:
y_pred[0:5]

764     10110.004883
887      5877.151367
890     28035.298828
1293     9675.225586
259     33566.500000
Name: charges, dtype: float32

In [54]:
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -4498.2533228689435
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -4498.2533228689435,
    "mean_squared_error": -20234282.956701495,
    "mean_absolute_error": -2478.135858821178,
    "r2": 0.8619241156846388,
    "pearsonr": 0.9285930727865682,
    "median_absolute_error": -1333.3959310546877
}


Predictions:  
 764     10110.004883
887      5877.151367
890     28035.298828
1293     9675.225586
259     33566.500000
            ...     
644     10440.750000
602     13730.225586
731     11534.818359
321      7549.561035
479      4811.167480
Name: charges, Length: 442, dtype: float32


In [55]:
save_model_predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-4440.935447,-5173.916806,0.038974,0.006997,0.359774,0.038974,0.006997,0.359774,1,True,3
1,CatBoost,-4454.783224,-5048.357888,0.008990,0.003996,17.870932,0.008990,0.003996,17.870932,1,True,6
2,WeightedEnsemble_L2,-4498.253323,-4965.510788,0.238857,0.100936,26.278721,0.010994,0.000999,0.509684,2,True,12
3,NeuralNetFastAI,-4511.273966,-5444.887902,0.053435,0.017989,2.377528,0.053435,0.017989,2.377528,1,True,8
4,LightGBM,-4569.536960,-5040.531574,0.014994,0.007997,0.448720,0.014994,0.007997,0.448720,1,True,4
5,XGBoost,-4661.412208,-5164.956518,0.046266,0.013992,0.669585,0.046266,0.013992,0.669585,1,True,9
6,ExtraTreesMSE,-4744.929210,-5194.262922,0.154357,0.071955,0.804503,0.154357,0.071955,0.804503,1,True,7
7,NeuralNetTorch,-4785.998760,-5596.321460,0.029985,0.012991,6.627894,0.029985,0.012991,6.627894,1,True,10
8,RandomForestMSE,-4795.645634,-5103.436805,0.173894,0.074953,0.821491,0.173894,0.074953,0.821491,1,True,5
9,LightGBMLarge,-5017.058130,-5337.109204,0.046968,0.006997,0.953408,0.046968,0.006997,0.953408,1,True,11


In [56]:
test_data["predicted_charges"] = y_pred

In [57]:
test_data.head()

,age,sex,bmi,children,smoker,region,charges,predicted_charges
764,45,female,25.175,2,no,northeast,9095.06825,10110.004883
887,36,female,30.020,0,no,northwest,5272.17580,5877.151367
890,64,female,26.885,0,yes,northwest,29330.98315,28035.298828
1293,46,male,25.745,3,no,northwest,9301.89355,9675.225586
259,19,male,31.920,0,yes,northwest,33750.29180,33566.500000


In [58]:
test_data["error_values"] = abs(test_data["charges"] - test_data["predicted_charges"])

In [59]:
test_data.head()

,age,sex,bmi,children,smoker,region,charges,predicted_charges,error_values
764,45,female,25.175,2,no,northeast,9095.06825,10110.004883,1014.936633
887,36,female,30.020,0,no,northwest,5272.17580,5877.151367,604.975567
890,64,female,26.885,0,yes,northwest,29330.98315,28035.298828,1295.684322
1293,46,male,25.745,3,no,northwest,9301.89355,9675.225586,373.332036
259,19,male,31.920,0,yes,northwest,33750.29180,33566.500000,183.791800
